In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.callbacks import ModelCheckpoint
import os.path
from keras.preprocessing.image import ImageDataGenerator

def fixed_generator(generator):
    for batch in generator:
        yield (batch, batch)

img_width, img_height = 256, 256

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
input_img = Input(shape=input_shape)

# Block 1
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(input_img)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), name='block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), name='block2_pool')(x)

# Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = MaxPooling2D((2, 2), name='block3_pool')(x)

# Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = MaxPooling2D((2, 2), name='block4_pool')(x)

# Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
encoded = MaxPooling2D((2, 2), name='block5_pool')(x)

x = UpSampling2D((2, 2))(encoded)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decode_block5_conv3')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decode_block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decode_block5_conv1')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decode_block4_conv3')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decode_block4_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decode_block4_conv1')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decode_block3_conv3')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decode_block3_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decode_block3_conv1')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='decode_block2_conv2')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='decode_block2_conv1')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='decode_block1_conv2')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='decode_block1_conv1')(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
autoencoder = Model(input_img, decoded)
weights_file="/Users/pavelgulaev/Desktop/cae_vgg16.best.hdf5"
if (os.path.isfile(weights_file)):
    autoencoder.load_weights(weights_file)
autoencoder.compile(optimizer='adadelta', loss='mse')

Определение ошибок декодирования обученного автокодировщика

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import collections
scores = {}
validation_shock_data_dir = '/Users/pavelgulaev/Desktop/Диплом/Шок-картинки/shock_pics2'
batch_size = 1
shock_pics_count = 240
random_pics_count = 320
validation_random_data_dir = '/Users/pavelgulaev/Desktop/Диплом/Шок-картинки/random_pics2'
test_datagen = ImageDataGenerator(rescale=1. / 255)
validation_generator = test_datagen.flow_from_directory(
    validation_shock_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None)
for i in xrange(0,shock_pics_count):
    img = next(validation_generator)
    scores[autoencoder.evaluate(img,img,batch_size=1)]=1

test_datagen = ImageDataGenerator(rescale=1. / 255)
validation_generator = test_datagen.flow_from_directory(
    validation_random_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None)
for i in xrange(0,random_pics_count):
    img = next(validation_generator)
    scores[autoencoder.evaluate(img,img,batch_size=1)]=0
oscores = collections.OrderedDict(sorted(scores.items()))

Алгоритм нахождения барьерного порога, в данном коде для CAE с использованием VGG16.

In [ ]:
tp = 0
tn = random_pics_count
fp = 0
fn = shock_pics_count
best_threshold = 0.0
f1 = 0.0
best_f1 = 0.0
for k, v in oscores.items():
    if (v==1):
        tp+=1
        fn-=1
    if (v==0):
        fp+=1
        tn-=1
    if (tp==0):
        continue
    precision = 1.0 * tp / (tp+fp)
    recall = 1.0 * tp / (tp+fn)
    f1 = 2 * precision * recall / (precision + recall)
    if (f1 > best_f1):
        best_f1 = f1
        best_threshold = k
print best_f1, best_threshold